In [4]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

import numpy as np
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [5]:
print(tf.__version__)

2.5.0


In [6]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])


In [7]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [8]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())


text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [9]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64


In [10]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [12]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'Two things haunt you throughout L\'intrus (The Intruder): who\'s the intruder and is it a movie or a dream you\'re watching? The ending is so shocking that for a while you\'re at a loss for an answer to either of those questions. The intruder pops up as different characters, different men in different circumstances who don\'t belong in the scene, so they\'re expelled from it, kindly or brutally, but often without emotional involvement. The main character, Louis, is a contemptible man. He\'s got rough ways, some mean job and no heart. He needs one and goes after it. He has a heart transplanted and afterwards decides to start a new life. Can this man succeed in his quest for redemption? A guy like that could cut your throat at the drop of a hat. You know it but Claire Denis doesn\'t encourage you to judge him. Occasionally, there\'s a young Russian woman -a beautiful girl who seems to inhabit someplace between heaven and earth - who does judge him. She may even punish him. But

In [13]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))


In [14]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]


array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [15]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[105, 180,   1, ...,   0,   0,   0],
       [ 51,   4,  18, ...,   0,   0,   0],
       [298,   1,   7, ...,   0,   0,   0]])

In [16]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()


Original:  b'Two things haunt you throughout L\'intrus (The Intruder): who\'s the intruder and is it a movie or a dream you\'re watching? The ending is so shocking that for a while you\'re at a loss for an answer to either of those questions. The intruder pops up as different characters, different men in different circumstances who don\'t belong in the scene, so they\'re expelled from it, kindly or brutally, but often without emotional involvement. The main character, Louis, is a contemptible man. He\'s got rough ways, some mean job and no heart. He needs one and goes after it. He has a heart transplanted and afterwards decides to start a new life. Can this man succeed in his quest for redemption? A guy like that could cut your throat at the drop of a hat. You know it but Claire Denis doesn\'t encourage you to judge him. Occasionally, there\'s a young Russian woman -a beautiful girl who seems to inhabit someplace between heaven and earth - who does judge him. She may even punish him. B

In [22]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model2 = tf.keras.Sequential([
    encoder, 
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


In [23]:
print([layer.supports_masking for layer in model2.layers])


[False, True, True, True, True]


In [24]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model2.predict(np.array([sample_text]))
print(predictions[0])


[0.0151789]


In [25]:
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [26]:
history = model2.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)


Epoch 1/10
391/391 [==============================] - 448s 1s/step - loss: 0.6871 - accuracy: 0.5000 - val_loss: 0.6584 - val_accuracy: 0.4917
Epoch 2/10
391/391 [==============================] - 507s 1s/step - loss: 0.4274 - accuracy: 0.7909 - val_loss: 0.3634 - val_accuracy: 0.8302
Epoch 3/10
391/391 [==============================] - 510s 1s/step - loss: 0.3357 - accuracy: 0.8544 - val_loss: 0.3360 - val_accuracy: 0.8417
Epoch 4/10
391/391 [==============================] - 513s 1s/step - loss: 0.3195 - accuracy: 0.8636 - val_loss: 0.3269 - val_accuracy: 0.8516
Epoch 5/10
346/391 [=========================>....] - ETA: 1:04 - loss: 0.3130 - accuracy: 0.8671

KeyboardInterrupt: 